In [1]:
import scrapy
import json
from scrapy.crawler import CrawlerProcess
from scrapy.spiders import CrawlSpider
from scrapy.utils.project import get_project_settings
import pandas as pd
from pandas import DataFrame,Series

In [2]:
from scrapy import optional_features
optional_features.remove('boto')

In [3]:
import string

In [4]:
companies = pd.read_pickle('companies.pkl')

In [5]:
companies = list(companies['name'])

In [6]:
companies_l = map(lambda x:x.split(),companies)

In [7]:
nbw_data = DataFrame(columns=['cname','post','title'])

In [8]:
nbw_data

,cname,post,title


In [10]:
companies[105]

'zipdial'

In [10]:
title = ''
urls = ''
post_text = []
max_pages = 2
paras = []
class NBWItem(scrapy.Item):
    # define the fields for your item here like:
    cname = scrapy.Field()
    title = scrapy.Field()
    post = scrapy.Field()
    
class NBWSpider(CrawlSpider):
    global max_pages
    name = "nbwposts"
    allowed_domains = ["www.nextbigwhat.com"]
    start_urls = []
    for c in companies[100:106]:
        for page in range(1,20):
            start_urls.append('http://www.nextbigwhat.com/page/'+str(page)+'/?s='+c)
    
    def process_title(self,t):
        exclude = set(string.punctuation)
        t = ''.join(ch if ch not in exclude else ' ' for ch in list(t))
        t = t.strip()
        t = t.lower()
        t = t.split()
        return t
    
    def get_cname(self,title):
        cname = []
        for word in title:
            for comp in companies_l:
                if word in comp:
                    cname.append(comp)
        return cname
        
    
    def getURLS(self,title):
        url_vec = []
        for t in title:
            t = self.process_title(t)
            uv = 0
            for word in t:
                for comp in companies_l:
                    if word in comp:
                        uv=1
                        #print(t)
            url_vec.append(uv)
        return url_vec
    
    def process_paras(self,para):
        p2 = para
        o_index = -1
        for i,cha in enumerate((para)):
            if cha == '<':
                o_index = i
            elif cha == '>' and o_index!=-1:
                del p2[o_index:i+1]
                o_index = -1
        return ''.join(p2)

    def parse(self, response):
            global title
            global urls
            title = response.xpath('//h2[@class="entry-title h3"]/a/text()').extract()
            title = [x.strip().encode('ascii','replace') for x in title]
            if len(title[0])!=0:                
                urls = response.xpath('//h2[@class="entry-title h3"]/a/@href')
                url_vec = self.getURLS(title)
                for i in range(len(url_vec)):
                    if url_vec[i]==1:
                        url = urls[i].extract()
                        yield scrapy.Request(url, callback=self.parse_dir_contents)
 


    def parse_dir_contents(self, response):
        global paras
        global post_text
        global nbw_data
        item = NBWItem()
        t = response.xpath('//h1[@class="entry-title h1"]/text()').extract()
        #print(t)
        it = self.process_title(t[0].encode('ascii','replace'))
        item['title'] = it
        item['cname'] = self.get_cname(it)
        #print(item['cname'])
        paras = response.xpath('//div[@class="entry-content herald-entry-content"]//p')
        if len(paras)==0:
            paras = response.xpath('//div[@class="entry-content herald-entry-content"]//div')
        post_text = ' '
        for para in paras:
            post_text += self.process_paras(" ".join(para.xpath('text()').extract()))
            post_text += " "
        post_text = self.process_title(post_text.encode('ascii','replace'))
        item['post'] = post_text
        if len(item['cname'])!=0:
            nbw_data = nbw_data.append({'cname':item['cname'],'title':item['title'],'post':item['post']},
                                      ignore_index=True)
        



In [11]:
process = CrawlerProcess(get_project_settings())
process.crawl(NBWSpider)
process.start()

INFO:scrapy.utils.log:Scrapy 1.0.5 started (bot: scrapybot)
2016-03-26 18:47:43 [scrapy] INFO: Scrapy 1.0.5 started (bot: scrapybot)
INFO:scrapy.utils.log:Optional features available: ssl, http11
2016-03-26 18:47:43 [scrapy] INFO: Optional features available: ssl, http11
INFO:scrapy.utils.log:Overridden settings: {}
2016-03-26 18:47:43 [scrapy] INFO: Overridden settings: {}
INFO:scrapy.middleware:Enabled extensions: CloseSpider, TelnetConsole, LogStats, CoreStats, SpiderState
2016-03-26 18:47:44 [scrapy] INFO: Enabled extensions: CloseSpider, TelnetConsole, LogStats, CoreStats, SpiderState
INFO:scrapy.middleware:Enabled downloader middlewares: HttpAuthMiddleware, DownloadTimeoutMiddleware, UserAgentMiddleware, RetryMiddleware, DefaultHeadersMiddleware, MetaRefreshMiddleware, HttpCompressionMiddleware, RedirectMiddleware, CookiesMiddleware, ChunkedTransferMiddleware, DownloaderStats
2016-03-26 18:47:44 [scrapy] INFO: Enabled downloader middlewares: HttpAuthMiddleware, DownloadTimeoutMid

In [12]:
nbw_data = nbw_data.append(pd.read_pickle('nbw_note.pkl'),ignore_index=True)

In [13]:
nbw_data.to_pickle('nbw_note.pkl')

In [14]:
nbw_data

,cname,post,title
0,[[vserv]],"[mobile, advertising, industry, is, ramping, u...","[idg, ventures, invest, 3mn, in, mobile, ad, n..."
1,[[inmobi]],"[mobile, ad, network, has, raised, 200, millio...","[inmobi, raises, 200, million, from, softbank,..."
2,[[vserv]],"[a, recent, idc, report, suggested, that, ther...","[vserv, mobi, brings, social, sharing, and, a,..."
3,[[vserv]],"[mobile, advertising, exchange, vserv, mobi, h...","[vserv, adds, more, power, to, appwrapper, now..."
4,[[vserv]],"[mobile, marketing, platform, vserv, has, anno...","[vserv, launches, aqua, native, for, seamless,..."
5,[[vserv]],"[vserv, has, announced, introduction, of, powe...","[vserv, appwrapper, extends, one, click, app, ..."
6,[[vserv]],"[ad, network, vserv, has, raised, its, new, ro...","[us, based, fund, maverick, ventures, enters, ..."
7,[[vserv]],"[vserv, has, launched, audiencepro, platform, ...","[vserv, launches, audiencepro, platform, bring..."
8,[[webengage]],"[which, provides, customer, engagement, tools,...","[now, build, a, realtime, conversion, engine, ..."
9,[[webengage]],"[mumbai, based, customer, feedback, and, surve...","[webengage, launches, feature, to, target, cus..."


In [11]:
nbw_data = pd.read_pickle('nbw_note.pkl')

In [14]:
nbw_data['tit_uni'] = nbw_data['title'].apply(lambda x: ' '.join(x))

In [15]:
nbw_data.head()

,cname,post,title,tit_uni
0,[[vserv]],"[mobile, advertising, industry, is, ramping, u...","[idg, ventures, invest, 3mn, in, mobile, ad, n...",idg ventures invest 3mn in mobile ad network v...
1,[[inmobi]],"[mobile, ad, network, has, raised, 200, millio...","[inmobi, raises, 200, million, from, softbank,...",inmobi raises 200 million from softbank japan
2,[[vserv]],"[a, recent, idc, report, suggested, that, ther...","[vserv, mobi, brings, social, sharing, and, a,...",vserv mobi brings social sharing and a b testi...
3,[[vserv]],"[mobile, advertising, exchange, vserv, mobi, h...","[vserv, adds, more, power, to, appwrapper, now...",vserv adds more power to appwrapper now integr...
4,[[vserv]],"[mobile, marketing, platform, vserv, has, anno...","[vserv, launches, aqua, native, for, seamless,...",vserv launches aqua native for seamless integr...


In [20]:
nbw_data = nbw_data.ix[nbw_data['tit_uni'].drop_duplicates().index]

In [21]:
del nbw_data['tit_uni']

In [23]:
nbw_data.reset_index(inplace=True)
del nbw_data['index']

In [24]:
nbw_data.to_pickle('nbw_posts.pkl')